# Freight Router Cookbook\n\nEnd-to-end examples for route APIs, artifacts, signatures, and ETA drift tooling.

In [ ]:
import json\nimport subprocess\nfrom pathlib import Path\n\nimport requests\n\nBASE_URL = "http://localhost:8000"\nBASE_URL

In [ ]:
requests.get(f"{BASE_URL}/health", timeout=30).json()

In [ ]:
route_payload = {\n    "origin": {"lat": 52.4862, "lon": -1.8904},\n    "destination": {"lat": 51.5072, "lon": -0.1276},\n    "vehicle_type": "rigid_hgv",\n    "scenario_mode": "no_sharing",\n    "weights": {"time": 1, "money": 1, "co2": 1},\n    "departure_time_utc": "2026-02-12T08:30:00Z",\n    "pareto_method": "epsilon_constraint",\n    "epsilon": {"duration_s": 20000, "monetary_cost": 5000, "emissions_kg": 5000}\n}\nroute_data = requests.post(f"{BASE_URL}/route", json=route_payload, timeout=60).json()\nroute_data["selected"]["metrics"]

In [ ]:
compare_payload = {\n    "origin": route_payload["origin"],\n    "destination": route_payload["destination"],\n    "vehicle_type": route_payload["vehicle_type"],\n    "weights": route_payload["weights"],\n    "max_alternatives": 5\n}\ncompare_data = requests.post(f"{BASE_URL}/scenario/compare", json=compare_payload, timeout=90).json()\ncompare_data["deltas"]

In [ ]:
run_id = compare_data["run_id"]\nscenario_manifest = requests.get(f"{BASE_URL}/runs/{run_id}/scenario-manifest", timeout=30).json()\nscenario_signature = requests.get(f"{BASE_URL}/runs/{run_id}/scenario-signature", timeout=30).json()\n\nunsigned_payload = dict(scenario_manifest)\nunsigned_payload.pop("signature", None)\nverify_data = requests.post(\n    f"{BASE_URL}/verify/signature",\n    json={"payload": unsigned_payload, "signature": scenario_signature["signature"]["signature"]},\n    timeout=30,\n).json()\nverify_data

In [ ]:
tmp = Path("backend/out/analysis")\ntmp.mkdir(parents=True, exist_ok=True)\ncsv_path = tmp / "notebook_eta_samples.csv"\ncsv_path.write_text(\n    "trip_id,predicted_eta_s,observed_eta_s\\n"\n    "T1,900,960\\n"\n    "T2,1200,1110\\n",\n    encoding="utf-8",\n)\ncmd = [\n    "uv",\n    "run",\n    "python",\n    "scripts/check_eta_concept_drift.py",\n    "--input-csv",\n    str(csv_path),\n    "--out-dir",\n    "out",\n]\nresult = subprocess.run(cmd, cwd="backend", capture_output=True, text=True, check=True)\njson.loads(result.stdout)